## Introduction

* categorical variable은 제한된 값을 사용함
    * 얼마나 자주 아침을 먹나요?에 대한 대답으로 "Never", "Rarely", "Most days", "Every day"가 있을 때, 이 데이터는 categorical하다고 함
    * 당신의 차 브랜드는 무엇인가요?에 대답으로 "Honda", "Toyota", "Ford"가 있을 때, 이 데이터 또한 categorical
* 이러한 데이터를 전처리없이 머신 러닝 모델에 사용하려고 하면 에러가 발생함

<br>

## Three Approaches

#### 1. Drop Categorical Variables

* 데이터셋에서 categorical variables(컬럼) 제거하기
    * categorical variables를 처리하는 가장 쉬운 방법
* 컬럼이 유용한 정보를 포함하지 않을 때 사용

<br>

#### 2. Label Encoding

* 고유한 값에 따른 정수 할당하기

<img src="../images/3/LabelEncoing.png">

* "Every day"==3, "Never"==0, "Rarely"==1, "Most days"==2를 할당함
* 위의 예제에서는 의미대로 순서를 매겼지만("Never"==0), 모든 categorical variables가 명확한 순서를 갖지는 않음
    * but, tree 기반의 모델(ex. decision tree, random forests)은 순서가 있는 label encoding이 더 좋을 수 있음
    
<br>

#### One-Hot Encoding

* 원래 데이터의 한 컬럼에 존재하는 카테고리를 이름으로 갖는 새 컬럼 추가하기
* 원래 카테고리가 새로 추가된 컬럼에 해당하면 1, 해당하지 않으면 0

<img src="../images/3/One-HotEncoding.png">

* "Red", "Yellow", "Green" 컬럼을 새롭게 추가하고, 동일한 row에서 Color 컬럼의 데이터가 속하는 컬럼에 1을 할당
    * ex. 첫번째 row의 "Red"는 새로 추가된 Red 컬럼에서 1이고, 나머지 컬럼에서는 0값을 가짐
* label encoding과 대조적으로, one-hot encoding은 카테고리의 순서를 지정하지 않음
    * one-hot encoding은 순서가 명확하지 않은 categorical datad에서 잘 작동함(ex. Red는 Yellow 보다 큰 것이 아님)
    * 이러한 변수를 **nominal variables**하고 함(명목형 변수)
* **일반적으로 한 컬럼에 너무 많은 카테고리(15개 이상)가 있을 때는 수행하지 않음**

<br>

## Example

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

X_train.head()

/Users/daseul/anaconda2/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


<br>

* 학습 데이터에서 모든 categorical variables 얻기
* 컬럼이 `object` 타입(dtype)이라는 것은 컬럼이 text data를 포함하고 있다는 것을 나타냄

In [2]:
## Get list of categorical variables ##
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


<br>

#### Define Function to Measure Quality of Each Approach

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## Function for comparing different approaches ##
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    
    return mean_absolute_error(y_valid, preds)

<br>

#### Score from Approach 1 (Drop Categorical Variables)

* `select_dtypes()`를 사용해 `object` 컬럼을 제거

In [4]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Dro pcategorical variables): ")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Dro pcategorical variables): 
175703.48185157913


#### Score from Approach 2 (Label Encoding)

* scikit-learn의 class `LabelEncoder`
* 각 컬럼별로 label encoding 수행

In [5]:
from sklearn.preprocessing import LabelEncoder

## Make copy to avoid changning original data ##
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

## Apply label encoder to each column with categorical data ##
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])
    
print("MAE from Approach 2 (Label Encoding): ")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding): 
165936.40548390493


* 한 컬럼내의 각 카테고리에 랜덤하게 값 부여
* 간단하게 값을 부여할 수도 있지만, 더 좋은 성능을 얻기 위해 더 나은(유용한) 값을 부여할 수 있음

<br>

#### Score from Approach 3 (One-Hot Encoding)

* scikit-learn이 class `OneHotEncoder`
* 이 동작을 커스터마이즈할 수 있게 하는 수 많은 파라미터가 있음
    * `handle_unknown='ignore'`: default='error'. validation data가 training data에서 나타나지 않은 클래스를 포함할 때 에러를 무시
        transform을 수행하는 동안 알수없는 카테고리를 만나면, 해당 값에 대한 one-hot encoding 결과는 0이 됨(inverse transform하면 None)
    * `sparse=False`: default=True. encoding된 컬럼을 sparse matrix(희소행렬. 대부분의 값이 0인 행렬) 대신 numpy array로 반환

In [13]:
from sklearn.preprocessing import OneHotEncoder

## Apply one-hot encoder to each column with categorical data ##
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

## One-hot encoding removed index; put it back ##
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

## Remove categorical columns (will replace with one-hot encoding) ##
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

## Add one-hot encoded columns to numerical features ##
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding): ")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding): 
166089.4893009678


<br>

## Which approach is best?

* 이 경우에선 컬럼을 전부 제거(Approach 1)하는 방법의 성능이 가장 좋지 않음
* 나머지 두 방법은 error가 비슷(의미 있는 장점 X)
* 일반적으로는 one-hot encoding(Approach 3)이 가장 좋은 성능을 내고, 컬럼을 전부 제거하는 방법의 성능이 가장 좋지 않음(경우에 따라 다르기도 함)